### Generating the MnAu2 cell structure inputs for the QE calculations

Read in an initial .cif file (6-atoms) and expand to 54 atoms cell by translating in the x,y directions to increase the number of Mn atoms. This allows for more magnetic patterns in the Mn layers. Save to file as a .cif. 
The patterns matrix and coordinateis saved to file are used to generate the input for calculating configuation energies.
Total configuration energies and the pattern matrix are input for the exchange model to calculate the exchange parameters. 


In [1]:
from pathlib import Path
from sys import stdout
import numpy as np
import pandas as pd
import itertools
from pymatgen import units
from pymatgen.transformations.standard_transformations import SupercellTransformation
import sympy
from scipy.constants import Boltzmann, electron_volt
from sympy import symbols
from sympy.vector import CoordSys3D

import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression


from ruamel.yaml import YAML
yaml = YAML()

from neighbormodels.structure import from_file
from neighbormodels.neighbors import count_neighbors
from neighbormodels.interactions import build_model

kB = 1000 * Boltzmann / electron_volt

pd.set_option("display.colheader_justify", "left")
pd.set_option("display.html.border", 0)
html_table_style = {"selector": "th", "props": [("text-align", "left")]}

### Starting with 6 atom cell:

In [2]:
cif_filepath = "data/mnau2-6.cif"
structure_mnau26 = from_file(structure_file=cif_filepath)
print(structure_mnau26)

Full Formula (Mn2 Au4)
Reduced Formula: MnAu2
abc   :   3.394839   3.394839   8.880636
angles:  90.000000  90.000000  90.000000
Sites (6)
  #  SP      a    b         c
---  ----  ---  ---  --------
  0  Mn    0    0    0
  1  Au    0.5  0.5  0.840319
  2  Au    0.5  0.5  0.159681
  3  Mn    0.5  0.5  0.5
  4  Au    0    0    0.340319
  5  Au    0    0    0.659681


### Getting the neighbor information to calculate the exchange parameters:

Magnetic structure read in from the 24 atom Mn8Au16 structure with determined unique (collinear) patterns.

In [155]:
cif_filepath = "data/mn8au16-collinear.cif"
structure_mnau2_24 = from_file(structure_file=cif_filepath)
#print(structure_mnau2_24)

/home/ssilayi/anaconda3/lib/python3.6/site-packages/pymatgen/core/lattice.py:694: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  result = np.linalg.lstsq(q.T, p.T)[0].T


Read in and rearrange the pattern matrices to match the atomic sequence in the structure and create pattern matrix useable 
in the exchange model calculation.  

In [103]:
mn_layer=pd.read_csv('generate_input/mg_collinear_patterns', header=None)
au_layer = pd.DataFrame(np.zeros((2,18)))
collinear_patterns = pd.concat([mn_layer[0:1],au_layer,
                                mn_layer[1:2],au_layer,
                                mn_layer[2:3],au_layer,
                                mn_layer[3:4],au_layer,
                                mn_layer[4:5],au_layer,
                                mn_layer[5:6],au_layer,
                                mn_layer[6:7],au_layer,
                                mn_layer[7:8],au_layer], ignore_index=True)
c_types =[]
c_types.append("nm")
index = np.arange(1,len(collinear_patterns.columns))
for i in index:
    c_types.append("c-"+str(i))
    
collinear_patterns.columns = c_types
#collinear_patterns

### Calculate the neighbor energies from the magnetic patterns:

In [106]:

mnau2_neighbor_data = count_neighbors(cell_structure=structure_mnau2_24, r=8.8)
exchange_model = build_model(magnetic_patterns=collinear_patterns, neighbor_data=mnau2_neighbor_data)

#drop the Au atoms
exchange_model = exchange_model[exchange_model.columns.drop(list(exchange_model.filter(regex='Au')))]

exchange_model

/home/ssilayi/anaconda3/lib/python3.6/site-packages/pymatgen/core/lattice.py:694: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  result = np.linalg.lstsq(q.T, p.T)[0].T


,pattern,J1_MnMn,J2_MnMn,J3_MnMn,J4_MnMn,J5_MnMn,J6_MnMn,J7_MnMn,J8_MnMn
0,c-1,0.000000,0.666667,0.666667,0.000000,1.333333,0.000000,0.666667,0.000000
1,c-10,-0.333333,0.000000,0.333333,0.000000,-0.666667,-0.666667,0.333333,0.000000
2,c-11,-0.333333,0.000000,0.333333,0.000000,-0.666667,-0.666667,0.333333,0.000000
3,c-12,0.333333,0.000000,1.000000,0.000000,0.666667,0.666667,1.000000,0.000000
4,c-13,0.000000,-1.333333,0.000000,1.333333,0.000000,0.000000,0.000000,-0.666667
5,c-14,0.000000,0.000000,0.666667,-1.333333,-1.333333,0.000000,0.666667,0.000000
6,c-15,1.333333,1.333333,2.666667,1.333333,5.333333,2.666667,2.666667,0.666667
7,c-16,0.000000,-0.666667,0.666667,0.000000,-1.333333,0.000000,0.666667,0.000000
8,c-17,0.000000,0.000000,0.666667,-1.333333,-1.333333,0.000000,0.666667,0.000000
9,c-2,0.333333,0.000000,1.000000,0.000000,0.666667,0.666667,1.000000,0.000000


### Calculated energies for different patterns using QE:

In [113]:
#Calculated energies from QE for the different patterns are tabulated and read in
qe_energies = pd.read_csv('data/energy.out.collinear.csv', header=None)
qe_energies.columns = ["pattern", "num_sites", "total_energy" ]
#qe_energies

### Checking magnetic moments and total energies:

Pre-process: Check that magnetic moment is not lost and that energy values are unique for the different patterns 
within a tolerance of 0.1 meV

In [157]:
#read in magnetic moments and return any that don't have the expected total magnetic moment (18)
mg_moments = pd.read_csv('data/mg_moment.collinear',sep='\t', header=None)
mg_moments = round (abs(mg_moments/4.0)).sum(axis=0)
mg_moments = pd.concat([pd.DataFrame({'pattern':c_types}), pd.DataFrame({'total_moment':mg_moments})], axis=1)
#mg_moments
mg_moments[(mg_moments['total_moment'] != 8 ) & (mg_moments['pattern'] != "nm") ] #.head()

,pattern,total_moment


In [158]:
#Check that all energies are not within the tolerance value of each other
ryd_to_ev = 13.605693009 #eV 
tol = 0.0001 #eV
qe_energies = pd.read_csv('data/energy.out.collinear.csv', header=None)
qe_energies.columns = ["pattern", "num_sites", "total_energy" ]

#convert to eV
qe_energies["total_energy"] = qe_energies["total_energy"]*ryd_to_ev

for i in range(len(qe_energies)-1):
    for j in range(1+1,len(qe_energies)):
        if (i != j):
            diff = abs(qe_energies["total_energy"][i] - qe_energies["total_energy"][j])
            if (  diff  <= tol ):
                qe_energies["pattern"][j] = qe_energies["pattern"][i]
                #print (i,j, qe_energies["total_energy"][i], qe_energies["total_energy"][j], diff)
                
                
#drop repeating energies            
qe_energies = qe_energies.reset_index(drop=True).drop_duplicates(["pattern"], keep="first")           
 
#convert back to Rydberg
qe_energies["total_energy"] = qe_energies["total_energy"]/ryd_to_ev
qe_energies


/home/ssilayi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,pattern,num_sites,total_energy
0,nm,24,-14691.879405
1,c-1,24,-14693.951620
2,c-2,24,-14693.957454
3,c-3,24,-14693.970500
4,c-4,24,-14693.947722
5,c-5,24,-14693.946375
6,c-6,24,-14693.953194
7,c-7,24,-14693.949463
8,c-8,24,-14693.941122
9,c-9,24,-14693.932390


Calculated energies and values from neighbor model matched according to pattern type. 
The enegies are normalized relative to the "nm" type (spin 0).

In [160]:
model_matrix = qe_energies \
    .assign(energy = lambda x:
        (x["total_energy"] -
         np.float64(x.query("pattern == 'nm'").loc[:, "total_energy"])) *
        1000 * units.Ha_to_eV / x["num_sites"])  \
    .merge(exchange_model, on=["pattern"]) \
    .query("pattern != 'nm'") \
    .assign(J1 = lambda x: x["J1_MnMn"]) \
    .assign(J2 = lambda x: x["J2_MnMn"]) \
    .assign(J3 = lambda x: x["J3_MnMn"]) \
    .assign(J4 = lambda x: x["J4_MnMn"]) \
    .loc[:, ["pattern", "energy", "J1", "J2", "J3", "J4"]]

model_matrix

,pattern,energy,J1,J2,J3,J4
1,c-1,-2349.493462,0.000000,0.666667,0.666667,0.000000
2,c-2,-2356.107643,0.333333,0.000000,1.000000,0.000000
3,c-3,-2370.898982,0.666667,0.666667,1.666667,0.666667
4,c-4,-2345.072973,-0.666667,0.666667,0.333333,0.666667
5,c-5,-2343.546301,-0.333333,0.000000,0.333333,0.000000
6,c-6,-2351.277985,0.333333,0.000000,1.000000,0.000000
7,c-7,-2347.047612,0.000000,-0.666667,0.333333,0.666667
8,c-8,-2337.590545,0.000000,-0.666667,0.333333,0.666667
9,c-9,-2327.689478,0.000000,0.000000,0.666667,-1.333333
10,c-10,-2332.660783,-0.333333,0.000000,0.333333,0.000000


### Fitting to get final parameters:

Final matrix of energies and exchange model is used in a fit function to get the values of the J parameters.
2 results are compared, and values are in units of Rydberg.

In [161]:
smf_exchange_fit = smf.ols(data=model_matrix, formula="energy ~ J1 + J2 + J3 + J4").fit()
smf_exchange_parameters = pd.DataFrame(smf_exchange_fit.params, columns=["stats_models"])
#smf_exchange_parameters

In [162]:
lm = LinearRegression()
lm_exchange_fit = lm.fit(X=model_matrix[["J1", "J2", "J3", "J4"]], y=model_matrix["energy"])
lm_exchange_parameters = pd.DataFrame({
    "sklearn": [lm_exchange_fit.intercept_, 
                lm_exchange_fit.coef_[0], lm_exchange_fit.coef_[1],lm_exchange_fit.coef_[2],lm_exchange_fit.coef_[3]]},
    index=["Intercept", "J1", "J2", "J3", "J4"],
)
#lm_exchange_parameters

In [163]:
exchange_parameters = pd.concat([smf_exchange_parameters,lm_exchange_parameters], axis=1)
#save to file    
exchange_parameters.to_csv("data/mnau2_exchange_parameters.collinear.txt", sep=' ')
    

exchange_parameters

,stats_models,sklearn
Intercept,-2343.262191,-2343.262191
J1,-17.678685,-17.678685
J2,-8.688744,-8.688744
J3,-0.658163,-0.658163
J4,-12.271814,-12.271814
